<h1>Analysis of time series data with PolyTrend change-detection algorithm</h1>
<p>This script allows you to use Google Earth Engine API to import and analyze time series data. You can analyze a total of 1 000 000 pixels at once.</p>
<h3>Step 1. Import libraries below.</h3>

In [1]:
#import libraries for PolyTrend algorithm
import pandas as pd
import numpy as np
import numpy.linalg as lng
import numpy.polynomial.polynomial as poly
import scipy.stats as stats
from ipyleaflet import Map, basemaps, DrawControl, basemap_to_tiles, CircleMarker
try:
    import ee
except ImportError:
    raise ImportError("You either haven't installed or authenticated Earth Engine")
    
ee.Initialize()

<h4>Step 1a. Alternatively, to use your own data, upload a csv file to this Anaconda environment, enter the name of the file in the next cell and run the code. Time series needs to have at least 4 time steps to perform cubic fitting. First row should contain names of columns, including 'latitude' and 'longitude'. Otherwise, go to the next step.</h4>

In [ ]:
file_name = 'enter the name of you file here, as string, ending with .csv'
dataset= pd.read_csv(file_name)
dataset.groupby(['longitude', 'latitude'])
print('You should see the first few records. Do they look ok?')
print(dataset.head())

<h3>Step 2. Enter parameters:</h3>
<br>
<ul>
    <li>Statistical significance (alpha), the default value is 0.05.</li> 
    <li>Coordinates of the region of interest. If you don't know them use the map, mark a polygon. Only the coordinates of the last polygon drawn will be used.</li>
    <li>ID of the dataset you'd like to use. Check its ID <a href="https://developers.google.com/earth-engine/datasets/catalog/">here</a>. Enter as variable name_of_collection.</li>
    <li>Start and end dates as strings to determine date range.</li>
    <li>Name of the band you want to analyse. This is case sensitive so please check what band names the dataset has on Earth Engine website.</li>
    <li>Threshold for minimum of analyzed values, eg. for NDVI it could be 0.2 to exclude water bodies. Please check what range is offered by the sensor you are using.</li>
    <li>Nominal scale in meters of the projection to work in.</li>
</ul>

<h4>Make sure to use the same type of data as in the example below, string for collection name, dates and band name, alpha, ndvi_threshold and scale should be numerical. If you are using your own coordinates enter them as [longitude, latitude], uncomment the line.</h4>

<h4 style='color:red'>Change the right side of the equation keeping the same format and type of data as in the example</h4> 

In [39]:
alpha = 0.05
name_of_collection = 'NASA/GIMMS/3GV0'
startYear = 1982
endYear = 2006
band_name = 'ndvi'
# coords = [[11.944024, 52.30512], [11.922045, 52.197507], [11.966003, 52.224435], [11.944024, 52.30512]]
ndvi_threshold = 0.1 #values from 0.3 up will be considered
scale =8000
print('Ready to go to the next step.')

Ready to go to the next step.


<h3>Step 3. Create a map to select area of interest.</h3>

In [40]:
m = Map(
    center=(15.605401,18.697332),
    zoom=2
)
#enable drawing tools
draw_control = DrawControl()
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 1.0
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}

m.add_control(draw_control)
m


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…

In [41]:
print('coordinates that will be used: ', draw_control.last_draw['geometry']['coordinates'])


coordinates that will be used:  [[[-7.945283, 13.952549], [-8.121112, 5.558399], [-0.956081, 5.689601], [-0.956081, 13.824567], [-7.945283, 13.952549]]]


<h3>Step 4. Generate annual max NDVI time series.</h3><br>
<i>Please note: 
    <ul>
    <li>Only the last polygon marked will be used. If you are using a point, you have to change the geometry inside the script so that AOI = ee.Geometry.Point(coords)</li>
    <li>To generate maximum annual value instead of mean change the word 'mean' to 'max' in this line: _collection.filterDate(start_date, end_date).mean() </li>
    </ul>

In [42]:
coords = draw_control.last_draw['geometry']['coordinates']

AOI = ee.Geometry.Polygon(coords)

#The below code is adapted from Tylere on Github: https://gist.github.com/tylere/42e4acf883e18f5b8e331cfab8c91ab5

collection = ee.ImageCollection(name_of_collection).select(band_name).filterBounds(AOI)
collection = collection.filterDate(ee.Date.fromYMD(startYear, 1, 1), ee.Date.fromYMD(endYear, 1, 1).advance(1, 'year'))

#Create list of years
years = ee.List.sequence(startYear, endYear, 1)

def calculateAnnualMean(year_and_collection):
  # Unpack variable from the input parameter
    year_and_collection = ee.List(year_and_collection)
    year = ee.Number(year_and_collection.get(0))
    _collection = ee.ImageCollection(year_and_collection.get(1))
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
   
    return  _collection.filterDate(start_date, end_date).max().set('system:time_start', year)

# Create a list of year-collection pairs (i.e. pack the function inputs)
list_of_years_and_collections = years.zip(ee.List.repeat(collection, years.length()))

annualNdvi = ee.ImageCollection.fromImages(list_of_years_and_collections.map(calculateAnnualMean))
print(annualNdvi.size().getInfo())


geom_values = annualNdvi.getRegion(geometry=AOI, scale=scale)
geom_values_list = ee.List(geom_values).getInfo()
# Convert to a Pandas DataFrame.
header = geom_values_list[0]
data = pd.DataFrame(geom_values_list[1:], columns=header)
data['datetime'] = pd.to_datetime(data['time'], unit='ms', utc=True)
data.set_index('time')
data.groupby(['longitude', 'latitude'])

print(data)

25
        id  longitude   latitude    time   ndvi  \
0        0  -8.084838   5.569555  1982.0  0.755   
1        1  -8.084838   5.569555  1983.0  0.756   
2        2  -8.084838   5.569555  1984.0  0.777   
3        3  -8.084838   5.569555  1985.0  0.797   
4        4  -8.084838   5.569555  1986.0  0.821   
5        5  -8.084838   5.569555  1987.0  0.727   
6        6  -8.084838   5.569555  1988.0  0.758   
7        7  -8.084838   5.569555  1989.0  0.843   
8        8  -8.084838   5.569555  1990.0  0.837   
9        9  -8.084838   5.569555  1991.0  0.858   
10      10  -8.084838   5.569555  1992.0  0.706   
11      11  -8.084838   5.569555  1993.0  0.821   
12      12  -8.084838   5.569555  1994.0  0.766   
13      13  -8.084838   5.569555  1995.0  0.829   
14      14  -8.084838   5.569555  1996.0  0.853   
15      15  -8.084838   5.569555  1997.0  0.900   
16      16  -8.084838   5.569555  1998.0  0.817   
17      17  -8.084838   5.569555  1999.0  0.847   
18      18  -8.084838   5.56

<h4>Step 4a. Alternatively, save the time series for the polygon. It saves to the active Anaconda environment as time_series.csv.</h4>

In [6]:
data.to_csv('Chad_ts.csv')

<h3>Step 5. Run PolyTrend algorithm per pixel. It will take a while to complete, dependent on the size of your data set.</h3>
<p><i>Watch for a message saying 'Running this process ended successfully.' below the cell</i><p>

In [43]:
message = 'Please wait for a message that the process is completed...'
#definition of the PolyTrend algorithm
def PolyTrend(Y, alpha):
    X = range(1, len(Y)+1)
 
    #define function to find p value:
    def Pvalue(coef, df, A, Aprim, pn):
        #generate square residual
        part_res = np.dot(A, pn)-Y
        residual = np.dot(part_res.transpose(), part_res)
        #generate variance-covariance matrix
        VC = lng.inv(np.dot(Aprim, A))*residual/df
        #compute variance of the first coefficient
        VC1 = np.sqrt(VC[0,0])
        #compute t-statistic
        statistic = coef/VC1
        #compute p value
        p = stats.t.sf(np.abs(statistic), df)*2
        return p;
    
    def Plinear(X, Y):
        df1 = len(X)-2
        #generate Vandermonde matrix
        A1 = np.vander(X, 2)
        #generate transpose of the Vandermonde matrix
        Aprim1 = A1.transpose()
        p1 = np.dot(np.dot((lng.inv(np.dot(Aprim1, A1))), Aprim1), Y)
        coef1 = p1[0]
        Plin = Pvalue(coef1, df1, A1, Aprim1, p1)
        Slope = p1[0]
        Direction = np.sign(Slope)
        #Slope and Direction will be referred to Plin[1] and Plin[2] respectively in returned results
        return Plin, Slope, Direction;
    
    #degrees of freedom
    df3 = len(X)-4
    #generate Vandermonde matrix
    A3 = np.vander(X, 4)
    #generate transpose of the Vandermonde matrix
    Aprim3 = A3.transpose()
    #X=inv(A'*A)*A'*L - creating coefficients matrix:
    p3 = np.dot(np.dot((lng.inv(np.dot(Aprim3, A3))), Aprim3), Y)
    coef3 = p3[0]
    #compute p-value for cubic fit
    Pcubic = Pvalue(coef3, df3, A3, Aprim3, p3)
    #get roots of cubic polynomial
    coefs3 = ([p3[2], 2*p3[1], 3*p3[0]])
    roots3 = np.sort(poly.polyroots(coefs3))

    if (np.imag(roots3[0]) == 0 and np.imag(roots3[1])==0 and roots3[0] != roots3[1] and X[0] <= roots3[0] <= X[-1] and X[0] <= roots3[1] <= X[-1] and Pcubic < alpha):
        Plin = Plinear(X, Y)
        if (Plin[0] < alpha):
            Trend_type = 3
            Significance = 1
            Poly_degree = 3
        else:
            Trend_type = -1
            Significance = -1
            Poly_degree = 3
            return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
    else:
        df2 = len(X)-3
        A2 = np.vander(X, 3)
        Aprim2 = A2.transpose()
        p2 = np.dot(np.dot((lng.inv(np.dot(Aprim2, A2))), Aprim2), Y)
        coef2 = p2[0]
        Pquadratic = Pvalue(coef2, df2, A2, Aprim2, p2)
        coefs2 = ([p2[1], 2*p2[0]])
        roots2 = np.sort(poly.polyroots(coefs2))
        
        if (X[0] <= roots2 <= X[-1] and Pquadratic < alpha):
            Plin = Plinear(X, Y)
            if Plin[0] < alpha:
                Trend_type = 2
                Significance = 1
                Poly_degree = 2
            else:
                Trend_type = -1
                Significance = -1
                Poly_degree = 2
                return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
                
        else:
            Plin = Plinear(X, Y)
            if Plin[0] < alpha:
                Trend_type = 1
                Significance = 1
                Poly_degree = 1
            else:
                Trend_type = 0
                Significance = -1
                Poly_degree = 0
            return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];     
        return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
    return [Trend_type, Significance, Poly_degree, Plin[1], Plin[2]];
#end of PolyTrend definition

#establish how many images there are in the collection
list_of_images = data['id']
ids_of_images = []
for img_id in list_of_images:
    if img_id not in ids_of_images:
        ids_of_images.append(img_id)
        
n = len(ids_of_images)
print('number of images: ', n)
number_of_pixels = len(data) 
print('number of pixels analysed: ', number_of_pixels)

#make_Y function returns the results of the analysis for each individual pixel identified by its coordinates
def make_Y(dataset, alpha):
    PT_result = []
    #split the dataset into pixel time series
    for i in range(0, number_of_pixels, n):
        Y = dataset[i:i+n][band_name].values 
        #eliminate numbers lower than the threshold and any other values that are not numeric
        for val in Y:
            if val > ndvi_threshold and isinstance(val, (int,float)):
                try:
                    result = list(PolyTrend(Y, alpha))
                except:
                    result = ['unqualified', 'unqualified', 'unqualified', 'unqualified', 'unqualified']
            else:
                result = ['NA', 'NA', 'NA', 'NA', 'NA']
        #populate the empty PT_result list with values    
        pixel_long = dataset.at[i, 'longitude']
        pixel_lat = dataset.at[i, 'latitude']
        PT_result_header = ['longitude', 'latitude', 'trend type', 'significance', 'degree', 'slope', 'direction']
        PT_result.append([pixel_long, pixel_lat, result[0], result[1], result[2], result[3], result[4]])
    #create a data frame for displaying results on a map    
    image_frame = pd.DataFrame(PT_result[0:], columns=PT_result_header)
    return image_frame;
print(message)
final_result = make_Y(data, alpha)
pixels_to_display = len(final_result)

#accompanying block of code, needed for the next steps, placed here so that the user can conveniently move to creating maps
def assign_color(value):
    if value==-1:
        return 'gray'
    elif value ==0:
        return 'yellow'
    elif value ==1:
        return 'green'
    elif value == 2:
        return 'blue'
    elif value == 3:
        return 'red'
    elif value == 'unqualified':
        return 'violet'
    else:
        return 'black'

print('points produced: ', pixels_to_display)
message = 'Process finished successfully. You can save your data to a csv file or display on the maps below.'
print(message)

center = coords[0]

number of images:  25
number of pixels analysed:  284150
Please wait for a message that the process is completed...
points produced:  11366
Process finished successfully. You can save your data to a csv file or display on the maps below.


<h3>Step 6a. Save results to a csv file in this Anaconda environment.</h3>

In [44]:
final_result.to_csv('BurkinaFaso_max.csv')

<h4 style='color:red'>At the moment if you generate maps with more than 200 pixels, it might crash the program. Instead, you can download a csv file and display it using any desktop GIS that accepts csv files.</h4> 

<h2>Trend type map</h2>

In [17]:
center = coords[0][0][0], coords[0][0][1], 
#generate trend type map
m_trend = Map(
    center=center,
    zoom=7
)
for i in range(0, 200):
# for i in range(0, len(final_result)):  
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.fill_color = assign_color(final_result.at[i, 'trend type'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 
    m_trend.add_layer(pixel)
    
m_trend

[0.048325, 8.030584]


Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…

<h3>Legend:</h3>
<ul>
    <li><button style='background:gray'>gray</button> - concealed trend</li>
    <li><button style='background:yellow'>yellow</button> - no trend</li>
    <li><button style='background:green'>green</button> - linear trend</li>
    <li><button style='background:blue'>blue</button> - quadratic trend</li>
    <li><button style='background:red'>red</button> - cubic trend</li>
    <li><button style='background:violet'>violet</button> - unqualified (below threshold)</li>
    <li><button style='background:black; color:white'>black</button> - NaN</li>
    </ul>

<h2>Statistical significance map</h2>

In [19]:
m_significance = Map(
    center=center,
    zoom=7
)

# for i in range(0, 200):
for i in range(0, len(final_result)):
  
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.color = assign_color(final_result.at[i, 'significance'])
    pixel.fill_color = assign_color(final_result.at[i, 'significance'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 
    m_significance.add_layer(pixel)

m_significance

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'attribution': 'Map data (c) <a href…

<h3>Legend</h3>
<ul>
    <li><button style='background:gray'>gray</button> - statistically insignificant</li>
    <li><button style='background:green'>green</button> - statistically significant</li>
    <li><button style='background:violet'>violet</button> - unqualified (below threshold)</li>
    <li><button style='background:black; color:white'>black</button> - NaN</li>
            </ul>

<h2>Polynomial degree map</h2>

In [65]:
m_degree = Map(
    center=AOI_zoom,
    zoom=12
)
# for i in range(0, 200):
for i in range(0, len(final_result)):
        
    pixel = CircleMarker()
    pixel.location = (final_result.at[i, 'latitude'], final_result.at[i, 'longitude'])
    pixel.fill_color = assign_color(final_result.at[i, 'degree'])
    pixel.stroke = False
    pixel.radius = 5
    pixel.fill_opacity = 1.0 
    m_degree.add_layer(pixel)

m_degree

Map(basemap={'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors',…

<h3>Legend</h3>
<ul>
    <li><button style='background:yellow'>yellow</button> - no trend</li>
    <li><button style='background:green'>green</button> - linear trend</li>
    <li><button style='background:blue'>blue</button> - quadratic trend</li>
    <li><button style='background:red'>red</button> - cubic trend</li>
    <li><button style='background:violet'>violet</button> - unqualified (below threshold)</li>
    <li><button style='background:black; color:white'>black</button> - NaN</li>
</ul>
        